# EDA - Focos de Queimadas no Brasil (2019-2024)
## Etapa 2 - Análise Exploratória de Dados

Este notebook implementa a análise exploratória de dados para o projeto, alinhado ao documento LaTeX `docs/projeto_aplicado.tex`.

**Conteúdo:**
1. Carregamento e descrição dos dados
2. Estatísticas descritivas (posição, dispersão, CV)
3. Distribuições e visualizações
4. Séries temporais e sazonalidade
5. Detecção de anomalias e outliers

In [ ]:
# Configuração inicial
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Configurações de exibição
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

# Resolver diretórios
if '__file__' in globals():
    PROJECT_ROOT = Path(__file__).resolve().parents[1]
else:
    cwd = Path.cwd().resolve()
    PROJECT_ROOT = cwd.parent if cwd.name == 'notebooks' else cwd

DATA_DIR = PROJECT_ROOT / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
FIGS_DIR = PROJECT_ROOT / 'figs' / 'eda'

print(f"Diretório do projeto: {PROJECT_ROOT}")
print(f"Diretório de dados processados: {PROCESSED_DIR}")
print(f"Diretório de figuras: {FIGS_DIR}")

## 1. Carregamento dos Dados

Carregamos o arquivo Parquet consolidado gerado pelo script `src/pipeline_ingestao.py`.

In [ ]:
# Carregar dados consolidados
parquet_path = PROCESSED_DIR / 'focos_2019_2024.parquet'

if not parquet_path.exists():
    raise FileNotFoundError(
        f"Arquivo {parquet_path} não encontrado. "
        "Execute: python -m src.pipeline_ingestao"
    )

df = pd.read_parquet(parquet_path)
print(f"Registros carregados: {len(df):,}")
print(f"Colunas: {df.shape[1]}")
print(f"\nPeríodo: {df['date'].min().date()} a {df['date'].max().date()}")

In [ ]:
# Primeiras linhas
df.head(10)

In [ ]:
# Informações sobre as colunas
df.info()

## 2. Estatísticas Descritivas

### 2.1 Resumo das colunas

In [ ]:
# Estatísticas básicas para colunas numéricas
df.describe()

In [ ]:
# Valores ausentes
missing = df.isna().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'n_missing': missing,
    'pct_missing': missing_pct
})
missing_df[missing_df['n_missing'] > 0].sort_values('n_missing', ascending=False)

### 2.2 Distribuição por Bioma

In [ ]:
# Contagem por bioma
bioma_col = 'bioma' if 'bioma' in df.columns else None

if bioma_col:
    by_bioma = df[bioma_col].value_counts()
    print("Focos por Bioma:")
    print(by_bioma)
    print(f"\nTotal de biomas: {df[bioma_col].nunique()}")
else:
    print("Coluna 'bioma' não encontrada")

### 2.3 Distribuição por UF

In [ ]:
# Contagem por UF
uf_col = 'estado' if 'estado' in df.columns else ('uf' if 'uf' in df.columns else None)

if uf_col:
    by_uf = df[uf_col].value_counts().head(15)
    print("Top 15 UFs por número de focos:")
    print(by_uf)
else:
    print("Coluna de UF não encontrada")

### 2.4 Estatísticas de agregação diária

In [ ]:
# Agregar por dia
by_day = df.groupby('day').size().reset_index(name='focos')
by_day['day'] = pd.to_datetime(by_day['day'])

# Estatísticas de focos por dia
print("Estatísticas de focos por dia:")
print(f"  Média: {by_day['focos'].mean():.2f}")
print(f"  Mediana: {by_day['focos'].median():.2f}")
print(f"  Desvio padrão: {by_day['focos'].std():.2f}")
print(f"  Variância: {by_day['focos'].var():.2f}")
print(f"  Mínimo: {by_day['focos'].min()}")
print(f"  Máximo: {by_day['focos'].max()}")
print(f"  Q1 (25%): {by_day['focos'].quantile(0.25):.2f}")
print(f"  Q3 (75%): {by_day['focos'].quantile(0.75):.2f}")
print(f"  IQR: {by_day['focos'].quantile(0.75) - by_day['focos'].quantile(0.25):.2f}")
print(f"  CV: {(by_day['focos'].std() / by_day['focos'].mean() * 100):.2f}%")

## 3. Visualizações

### 3.1 Série temporal geral

In [ ]:
# Série temporal de focos por dia
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(by_day['day'], by_day['focos'], color='darkred', linewidth=0.8, alpha=0.7)
ax.set_title('Focos de Queimadas por Dia (2019-2024)', fontsize=14, fontweight='bold')
ax.set_xlabel('Data')
ax.set_ylabel('Número de focos')
ax.grid(True, alpha=0.3)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 3.2 Séries temporais por Bioma

In [ ]:
# Séries por bioma (usar a imagem gerada)
from IPython.display import Image, display

img_path = FIGS_DIR / 'series_bioma.png'
if img_path.exists():
    display(Image(filename=str(img_path)))
else:
    print(f"Figura não encontrada: {img_path}")
    print("Execute: python -m src.eda_utils")

### 3.3 Boxplot por Bioma

In [ ]:
# Boxplot por bioma
img_path = FIGS_DIR / 'boxplot_bioma.png'
if img_path.exists():
    display(Image(filename=str(img_path)))
else:
    print(f"Figura não encontrada: {img_path}")

### 3.4 Top 10 UFs

In [ ]:
# Top 10 UFs
img_path = FIGS_DIR / 'top10_uf.png'
if img_path.exists():
    display(Image(filename=str(img_path)))
else:
    print(f"Figura não encontrada: {img_path}")

### 3.5 Distribuição de focos diários (Histograma)

In [ ]:
# Histograma de focos por dia
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(by_day['focos'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_title('Distribuição de Focos Diários', fontsize=12, fontweight='bold')
ax.set_xlabel('Número de focos por dia')
ax.set_ylabel('Frequência')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 4. Sazonalidade

### 4.1 Focos por mês do ano

In [ ]:
# Agregar por mês (1-12)
if 'month' in df.columns:
    by_month = df.groupby('month').size().reset_index(name='focos')
    
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(by_month['month'], by_month['focos'], color='coral', alpha=0.8)
    ax.set_title('Focos por Mês (Agregado 2019-2024)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Mês')
    ax.set_ylabel('Total de focos')
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 
                        'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
    ax.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
else:
    print("Coluna 'month' não encontrada")

## 5. Detecção de Anomalias

### 5.1 Z-score robusto (MAD)

In [ ]:
# Carregar anomalias detectadas
anomalies_path = PROCESSED_DIR / 'anomalias_top.csv'

if anomalies_path.exists():
    anomalies = pd.read_csv(anomalies_path)
    print(f"Anomalias detectadas: {len(anomalies)}")
    print("\nTop 10 dias com maior número de focos (anomalias):")
    display(anomalies.head(10))
else:
    print(f"Arquivo {anomalies_path} não encontrado")
    print("Execute: python -m src.eda_utils")

### 5.2 Visualização de anomalias

In [ ]:
# Plotar série com anomalias destacadas
if anomalies_path.exists():
    anomalies = pd.read_csv(anomalies_path)
    anomalies['day'] = pd.to_datetime(anomalies['day'])
    
    fig, ax = plt.subplots(figsize=(14, 5))
    ax.plot(by_day['day'], by_day['focos'], color='gray', linewidth=0.8, alpha=0.6, label='Normal')
    ax.scatter(anomalies['day'], anomalies['focos'], color='red', s=30, alpha=0.7, 
               label='Anomalias', zorder=5)
    ax.set_title('Focos por Dia com Anomalias Destacadas', fontsize=14, fontweight='bold')
    ax.set_xlabel('Data')
    ax.set_ylabel('Número de focos')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("Anomalias não disponíveis")

## 6. Conclusões da AED

Esta análise exploratória demonstra:

1. **Volume de dados**: Mais de 2 milhões de registros consolidados de focos de queimadas entre 2019 e 2024
2. **Distribuição espacial**: Concentração em biomas específicos (Amazônia, Cerrado) e estados críticos
3. **Sazonalidade**: Padrão claro de aumento de focos nos meses de seca (jul-out)
4. **Anomalias**: Identificados dias com picos atípicos que merecem investigação adicional
5. **Qualidade dos dados**: Dados limpos com mínima presença de valores ausentes após tratamento

Os artefatos gerados (figuras e CSVs) estão referenciados no documento LaTeX `docs/projeto_aplicado.tex`.